In [17]:
%pip install -qU langchain langchain-openai langchain-community langchain-experimental pandas

Note: you may need to restart the kernel to use updated packages.


In [28]:
%pip install --upgrade --quiet  langchain langchain-community langchain-experimental

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain.llms import LlamaCpp
from langchain.agents import AgentType
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

#from langchain_core.prompts import ChatPromptTemplate
#from langchain_experimental.tools import PythonAstREPLTool

#from langchain_experimental.agents.agent_toolkits import create_csv_agent



path="7b_prod.gguf"
llm =  LlamaCpp(
        model_path=path,
        temperature=0.75,
        #max_tokens=2000,
        #top_p=1,
        #verbose=True,
 #       n_gpu_layers=n_gpu_layer,
 #       n_batch=n_batch,
        #n_ctx=2048,
        #f16_kv=True,
        )

In [2]:
llm =  LlamaCpp(
        model_path=path,
        temperature=0.75,
        #max_tokens=2000,
        #top_p=1,
        #verbose=True,
 #       n_gpu_layers=n_gpu_layer,
 #       n_batch=n_batch,
        #n_ctx=2048,
        #f16_kv=True,
        )

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from 7b_prod.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_lo

In [4]:
import numpy as np         # linear algebra
import pandas as pd        # data processing, CSV file I/O (e.g. pd.read_csv)

# Read the file "data.csv" and print the contents.
#!cat data/data.csv
df = pd.read_csv('data.csv', index_col=False,)

# Id column is redundant and not useful, we want to drop it
df.drop('id', axis =1, inplace=True)
#data.drop('Unnamed: 0', axis=1, inplace=True)

df.info()

engine = create_engine("sqlite:///diagnosis.db")
df.to_sql("diagnosis", engine, index=False)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                569 non-null    object 
 1   radius_mean              569 non-null    float64
 2   texture_mean             569 non-null    float64
 3   perimeter_mean           569 non-null    float64
 4   area_mean                569 non-null    float64
 5   smoothness_mean          569 non-null    float64
 6   compactness_mean         569 non-null    float64
 7   concavity_mean           569 non-null    float64
 8   concave points_mean      569 non-null    float64
 9   symmetry_mean            569 non-null    float64
 10  fractal_dimension_mean   569 non-null    float64
 11  radius_se                569 non-null    float64
 12  texture_se               569 non-null    float64
 13  perimeter_se             569 non-null    float64
 14  area_se                  5

ValueError: Table 'diagnosis' already exists.

In [6]:
df.diagnosis.unique()
diag_gr = df.groupby('diagnosis', axis=0)
pd.DataFrame(diag_gr.size(), columns=['# of observations'])

/tmp/ipykernel_2462922/3340726935.py:2: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  diag_gr = df.groupby('diagnosis', axis=0)


,# of observations
diagnosis,
B,357
M,212


In [7]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT count(*) FROM diagnosis WHERE diagnosis = 'M';")

sqlite
['diagnosis']


'[(212,)]'

In [9]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [10]:
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [11]:
agent_executor.invoke(
    "Count where the diagnosis is M?"
)



> Entering new SQL Agent Executor chain...


TypeError: Llama.__call__() got an unexpected keyword argument 'tools'